In [1]:
import numpy as np
import pandas as pd
import h5py
import csv
from DataGenerator import DataGenerator
from Models import Models
from LoadData import LoadData
from BaselineHelperFunctions import BaselineHelperFunctions
from RandomGridSearch import RandomGridSearch
import tensorflow as tf

Using TensorFlow backend.


In [2]:
csv_root = 'csv_folder_3_class'
full_data_csv, train_csv, val_csv, test_csv = LoadData(csv_root, isBalanced = True).getData()
data_gen = DataGenerator(csv_root, train_csv, val_csv, test_csv)
full_ds = data_gen.load_dataset(full_data_csv)
train_ds = data_gen.load_dataset(train_csv)
val_ds = data_gen.load_dataset(val_csv)
test_ds = data_gen.load_dataset(test_csv)


In [3]:
model_nr = 5
    
batch_size = 128
epochs = 100
learning_rate = 0.0001
opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
activation = 'tanh'
dropout_rate = 0.2
l1_r = 0.2
l2_r = 0.001
start_neurons = 512

test = False
num_classes = 3
detrend = False


num_ds, channels, timesteps = data_gen.get_trace_shape_no_cast(train_ds)

build_model_args = {'model_nr' : model_nr,
                    'input_shape' : (batch_size, channels, timesteps),
                    'num_classes' : num_classes,
                    'dropout_rate' : dropout_rate,
                    'activation' : activation,
                    'l2_r' : l2_r,
                    'l1_r' : l1_r,
                    'full_regularizer' : True,
                    'start_neurons' : start_neurons,
                    'filters' : 16,
                    'kernel_size' : 10,
                    'padding' : 'same'}
model = Models(**build_model_args).model

model_args = {'loss' : "categorical_crossentropy",
              'optimizer' : opt,
              'metrics' : ["accuracy","MSE",
                           tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
                           tf.keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None)]}
model.compile(**model_args)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (128, 3, 16)              960176    
_________________________________________________________________
batch_normalization_1 (Batch (128, 3, 16)              64        
_________________________________________________________________
flatten_1 (Flatten)          (128, 48)                 0         
_________________________________________________________________
dense_1 (Dense)              (128, 3)                  147       
Total params: 960,387
Trainable params: 960,355
Non-trainable params: 32
_________________________________________________________________
